In [1]:
import os
import sys
import pandas as pd
import numpy as np
import folium
import io
from PIL import Image

from helpers import (
    peform_mds_division,
)

path = os.path.abspath("../spatial-awareness/geocoding/")
sys.path.append(path)
from geocoding import GeoCoding

%load_ext autoreload
%autoreload 2

In [2]:
cities = pd.read_pickle('cities.pkl')
cities_list = list(set(cities.a_name.to_list()))

gc = GeoCoding()

coods = []
coods_dic = {}
for each in cities_list:
    lat, lng, _, _ , _ = gc.get_lat_lng(each)
    coods.append([lng, lat])
    coods_dic[each] = [lat, lng]
    
coods = np.array(coods)

dis_disparity_df = pd.DataFrame(0.0,index=cities_list, columns=cities_list)

for i, each in dis_disparity_df.iterrows():
    for c in cities_list:
        if i == c:
            continue
        d = cities.loc[(cities.a_name == i) & (cities.b_name == c)]
        each[c] = d['distance'].to_list()[0]

vals = peform_mds_division(
    disparity_df=dis_disparity_df,
    coods_dic=coods_dic,
    metric=True,
    asymmetric=False,
    similarity_measure_used=False
)
df_1 = pd.DataFrame(vals)
df_1.rename(columns={
    'pred_lat': 'lat_dis',
    'pred_lng': 'lng_dis',
    'err': 'err_dis',
}, inplace=True)

pred_dis_disparity_df = pd.DataFrame(0.0,index=cities_list, columns=cities_list)

for i, each in pred_dis_disparity_df.iterrows():
    for c in cities_list:
        if i == c:
            continue
        d = cities.loc[(cities.a_name == i) & (cities.b_name == c)]
        each[c] = d['predicted_dis'].to_list()[0]

vals_pred = peform_mds_division(
    disparity_df=pred_dis_disparity_df,
    coods_dic=coods_dic,
    metric=True,
    asymmetric=True,
    similarity_measure_used=False
)
df_2 = pd.DataFrame(vals_pred)
df_2.rename(columns={
    'pred_lat': 'lat_pred_dis',
    'pred_lng': 'lng_pred_dis',
    'err': 'err_pred_dis',
}, inplace=True)

df = pd.merge(df_1, df_2, on=['name', 'lat', 'lng'])

def get_all_coods(name, df=df):
    t = df.loc[df.name==name]
    return {
        'original': [t['lat'].item(), t['lng'].item()],
        'dis': [t['lat_dis'].item(), t['lng_dis'].item()],
        'pred_dis': [t['lat_pred_dis'].item(), t['lng_pred_dis'].item()],
    }


In [45]:

def add_markers_to_map(map, name, fill_opacity_dis=False, fill_opacity_pred=False):
    coods = get_all_coods(name)
    folium.Marker(
        coods['original'],
        icon=folium.Icon(color='green', icon='circle', prefix='fa'),
    ).add_to(map)

    if fill_opacity_dis:
        folium.Marker(
            coods['dis'],
            icon=folium.Icon(color='blue', icon='circle', prefix='fa'),
            opacity=0.7,
        ).add_to(map)
    else:
        folium.Marker(
            coods['dis'],
            icon=folium.Icon(color='blue', icon='circle', prefix='fa'),
        ).add_to(map)

    if fill_opacity_pred:
        folium.Marker(
            coods['pred_dis'],
            icon=folium.Icon(color='red', icon='circle', prefix='fa'),
        ).add_to(map)
    else:
        folium.Marker(
            coods['pred_dis'],
            icon=folium.Icon(color='red', icon='circle', prefix='fa'),
        ).add_to(map)

def generate_map(name, save_name, fill_opacity_dis=False, fill_opacity_pred=False, fill_opacity_orig=False):
    zoom = 6
    m = folium.Map(
        location=get_all_coods(name)['original'],
        tiles="Stamen Toner",
        zoom_start=zoom,
    )

    add_markers_to_map(m, name, fill_opacity_dis, fill_opacity_pred, fill_opacity_orig=True)
    return m
    # img_data = m._to_png(10)
    # img = Image.open(io.BytesIO(img_data))
    # img.save(f'map_images/coods_preds-{save_name}.png')

In [47]:
# Albany, New York
name = 'Albany, New York'
generate_map(name, 'albany')

In [49]:
# Havre, Montana
name = 'Havre, Montana'
generate_map(name, 'havre')

In [53]:
# Fresno, California
name = 'Raleigh, North Carolina'
generate_map(name, 'fresno', fill_opacity_orig=True)

In [51]:
df.head()

,name,lat,lng,lat_dis,lng_dis,err_dis,lat_pred_dis,lng_pred_dis,err_pred_dis
0,"Bangor, Maine",44.80118,-68.77781,44.989853,-68.563321,26.956800,43.822867,-68.565534,110.019842
1,"Boston, Massachusetts",42.35843,-71.05977,42.318048,-71.047781,4.593163,42.433342,-71.284792,20.310115
2,"Eastport, Maine",44.90619,-66.98998,46.204894,-68.677339,195.420199,46.241721,-67.091507,148.646319
3,"Providence, Rhode Island",41.82399,-71.41283,41.704455,-71.480292,14.413122,42.117892,-72.094114,65.226894
4,"New Haven, Connecticut",41.30815,-72.92816,40.645577,-72.255445,92.842420,41.507036,-72.191537,65.433271


In [52]:
df.sort_values(by=['err_dis'])

,name,lat,lng,lat_dis,lng_dis,err_dis,lat_pred_dis,lng_pred_dis,err_pred_dis
39,"Raleigh, North Carolina",35.7721,-78.63861,35.786443,-78.664375,2.821176,37.339091,-79.887889,206.746038
75,"Salt Lake City, Utah",40.76078,-111.89105,40.732897,-111.854197,4.390306,40.906643,-113.190860,110.823212
1,"Boston, Massachusetts",42.35843,-71.05977,42.318048,-71.047781,4.593163,42.433342,-71.284792,20.310115
53,"Dallas, Texas",32.78306,-96.80667,32.746082,-96.778588,4.872529,32.391392,-97.686562,93.331785
51,"Nashville, Tennessee",36.16589,-86.78444,36.217505,-86.726617,7.736362,36.009349,-86.811206,17.536385
...,...,...,...,...,...,...,...,...,...
79,"Reno, Nevada",39.52963,-119.8138,39.450211,-118.371241,124.408861,38.361125,-112.356134,659.203051
23,"Wichita, Kansas",37.69224,-97.33754,36.614509,-96.740418,130.840668,36.781734,-94.263853,290.841519
65,"Havre, Montana",48.54278,-109.76298,49.601999,-110.757598,138.407268,48.184929,-107.776356,152.485357
2,"Eastport, Maine",44.90619,-66.98998,46.204894,-68.677339,195.420199,46.241721,-67.091507,148.646319
